In [1]:

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# input csv
ALL_DATA_csvpath = "../output_files/MEGAdf.csv"

#import pretty print
import pprint as pprint

#for sleep
import time

In [2]:
ALL_DATA_csv = pd.read_csv(ALL_DATA_csvpath,index_col=0)

In [3]:
# Set occupation here
ALL_DATA_df_collist_no_occ = [col for col in ALL_DATA_csv.columns.tolist() if "occupations" not in col]
ALL_DATA_df_occ_collist = [col for col in ALL_DATA_csv.columns.tolist() if "occupations" in col\
                             if "RANK" not in col]
global careerList
careerList={}
for key in range(1,len(ALL_DATA_df_occ_collist)+1):
    careerList[str(key)] = ALL_DATA_df_occ_collist[key-1]
     
    
def jbReq():
    for i in list(careerList.keys()):
        print(f"{i}: {careerList[i]}")
    global OCC_CHOICE
    OCC_CHOICE = input("Which career field are you looking for information on?  Please input the career ID #: ")
    try:
        careerList[OCC_CHOICE]
        print(f"Valid choice. Good luck in your new venture in {careerList[OCC_CHOICE]}")
    except:
        print("-----------------------------------------------------------------------")
        print("No career found with that ID # please reference the list and try again.")
        print("-----------------------------------------------------------------------")
        jbReq()
#################################################### 
jbReq() #Actually run the function now
ALL_DATA_df_collist_oneocc = ALL_DATA_df_collist_no_occ + [careerList[OCC_CHOICE]] + [careerList[OCC_CHOICE]+"_RANK"]

1: Architecture and engineering occupations
2: Arts, design, entertainment, sports, and media occupations
3: Building and grounds cleaning and maintenance occupations
4: Business and financial operations occupations
5: Community and social service occupations
6: Computer and mathematical occupations
7: Construction and extraction occupations
8: Educational instruction and library occupations
9: Food preparation and serving related occupations
10: Healthcare practitioners and technical occupations
11: Healthcare support occupations
12: Installation, maintenance, and repair occupations
13: Legal occupations
14: Life, physical, and social science occupations
15: Management occupations
16: Office and administrative support occupations
17: Personal care and service occupations
18: Production occupations
19: Protective service occupations
20: Sales and related occupations
21: Transportation and material moving occupations
Which career field are you looking for information on?  Please input t

In [4]:
# Limit the dataframe's occupation sections based on user choice.
USER_CHOICE_df = ALL_DATA_csv[ALL_DATA_df_collist_oneocc].reset_index(drop=True)

# Add the ranked column for the occupation to the overall score, and re-rank the overall score for overall rank.
USER_CHOICE_df["Overall_SCORE"] = USER_CHOICE_df["overall_SCORE"] + USER_CHOICE_df[careerList[OCC_CHOICE] +"_RANK"]
USER_CHOICE_df["Overall_RANK"] = USER_CHOICE_df["Overall_SCORE"].rank()

In [5]:
USER_CHOICE_df["OCC CHOICE"] = careerList[OCC_CHOICE]
USER_CHOICE_df["OCC VALUE"] = USER_CHOICE_df[careerList[OCC_CHOICE]]
USER_CHOICE_df["OCC VALUE R"]=USER_CHOICE_df[careerList[OCC_CHOICE] +"_RANK"]

In [6]:
# NOTE: Do not change any of the code in this cell
# {OCC CHOICE}{OCC VALUE}
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>City (Overall Rank)</dt><dd>{City Name},{stateabbrev} ({overall_RANK})</dd>
<dt>City Population (Rank)</dt><dd>{city_pop2019} ({city_pop2019_RANK})</dd>
<dt>Expected Hourly Wage for {OCC CHOICE} (Rank)</dt><dd>{OCC VALUE} ({OCC VALUE R})</dd>
<dt>% Change in Businesses Active since Jan 19 (Rank)</dt><dd>{merchants_all} ({merchants_all_RANK})</dd>
<dt>New Case Rate - Covid (Rank)</dt><dd>{new_case_rate} ({new_case_rate_RANK})</dd>
<dt>% Change in Job Postings since Jan 19 (Rank)</dt><dd>{bg_posts} ({bg_posts_RANK})</dd>
<dt>Avg. Cost of Living (Rank)</dt><dd>{Average Cost of Living } ({Average Cost of Living _RANK})</dd>
<dt>Avg. Nearby Park Rating (Rank)</dt><dd>{Park Score} ({Park Score_RANK})</dd>
<dt>Avg. Nearby Hospital Rating (Rank)</dt><dd>{Health score} ({Health score_RANK})</dd>

</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
ALL_DATA_info = [info_box_template.format(**row) for index, row in USER_CHOICE_df.iterrows()]
locations = USER_CHOICE_df[["lat", "lon"]]

In [9]:


# Import API key
#from api_keys import g_key

import sys
path = "/Users/matth/Desktop"
sys.path.insert(0, path)
from API_Keys import api_key
g_key= api_key["Google Maps Platform"]

# configure gmaps with api key
gmaps.configure(api_key=g_key)

fig = gmaps.figure()

#HEAT LAYER
heat_layer = gmaps.heatmap_layer(locations, weights=USER_CHOICE_df["overall_RANK"], 
                                 dissipating=False, max_intensity=USER_CHOICE_df["overall_RANK"].max(),
                                 point_radius=1)

# Add marker layer
marker_layer = gmaps.marker_layer(locations, \
                                 info_box_content=[f"{info}" for info in ALL_DATA_info])

# Add layer and Display figure
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


fig


Figure(layout=FigureLayout(height='420px'))

In [10]:
USER_CHOICE_df.sort_values(by = "OCC VALUE R")

,Park Score,Health score,City Name,stateabbrev,lat,lon,city_pop2019,merchants_all,new_case_rate,bg_posts,...,new_case_rate_RANK,overall_SCORE,overall_RANK,"Arts, design, entertainment, sports, and media occupations","Arts, design, entertainment, sports, and media occupations_RANK",Overall_SCORE,Overall_RANK,OCC CHOICE,OCC VALUE,OCC VALUE R
28,4.325000,3.505882,Washington,DC,38.91,-77.04,705749,-44.70%,15.30,0.16800,...,24.0,170.0,19.0,41.62,1.0,171.0,12.0,"Arts, design, entertainment, sports, and media...",41.62,1.0
0,4.347368,4.233333,Los Angeles,CA,34.05,-118.24,10039107,-33.30%,18.60,0.05670,...,29.0,141.5,10.0,39.53,2.0,143.5,10.0,"Arts, design, entertainment, sports, and media...",39.53,2.0
1,4.585000,3.784211,New York City,NY,40.71,-74.01,8336817,-42.00%,49.30,0.15300,...,51.0,118.0,5.0,39.42,3.0,121.0,4.0,"Arts, design, entertainment, sports, and media...",39.42,3.0
40,4.568421,3.254545,Oakland,CA,37.80,-122.27,1671329,-36.50%,7.99,0.17000,...,5.0,110.0,4.0,38.83,4.5,114.5,1.0,"Arts, design, entertainment, sports, and media...",38.83,4.5
20,4.435000,4.150000,San Francisco,CA,37.77,-122.42,881549,-51.20%,8.38,0.01150,...,6.0,130.0,8.0,38.83,4.5,134.5,6.0,"Arts, design, entertainment, sports, and media...",38.83,4.5
11,3.942105,4.614286,San Jose,CA,37.34,-121.89,1927852,-40.50%,12.30,0.09220,...,17.0,127.0,7.0,36.33,6.0,133.0,5.0,"Arts, design, entertainment, sports, and media...",36.33,6.0
8,4.106667,3.468750,Seattle,WA,47.61,-122.33,2252782,-25.90%,9.70,0.13600,...,10.0,179.5,23.0,35.40,7.0,186.5,18.0,"Arts, design, entertainment, sports, and media...",35.40,7.0
24,4.610526,4.233333,Boston,MA,42.36,-71.06,803907,-47.70%,23.20,0.33600,...,42.0,107.5,2.0,34.76,8.0,115.5,2.5,"Arts, design, entertainment, sports, and media...",34.76,8.0
23,4.247059,4.600000,Portland,OR,45.52,-122.68,812855,-39.70%,7.21,0.13600,...,3.0,130.5,9.0,32.35,9.0,139.5,8.0,"Arts, design, entertainment, sports, and media...",32.35,9.0
16,4.335294,4.000000,Charlotte,NC,35.23,-80.84,1110356,-27.00%,26.50,0.02380,...,46.0,231.0,42.5,31.65,10.0,241.0,34.0,"Arts, design, entertainment, sports, and media...",31.65,10.0
